In [11]:
from util.utils import *
import os

In [12]:
## load datasets
def loadDataframes():
    cat62 = None
    #cat62 = openDataframe("cat62_bronze.csv")
    espera = openDataframe("espera_bronze.csv")
    bimtra = openDataframe("bimtra_bronze.csv")
    metaf = openDataframe("metaf_bronze.csv")
    metar = openDataframe("metar_bronze.csv")
    tc_prev = openDataframe("tc-prev_bronze.csv")
    tc_real = openDataframe("tc-real_bronze.csv")
    return cat62, espera, bimtra, metaf, metar, tc_prev, tc_real

cat62, espera, bimtra, metaf, metar, tc_prev, tc_real = loadDataframes()

In [13]:
tc_real['aero'] = tc_real['aero'].apply(lambda x: 'SB' + x)

In [14]:
# MERGE BIMTRA - TC_REA

resultado = bimtra.merge(tc_real, left_on=['origem'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado = resultado[resultado.apply(lambda row: abs(row['dt_dep'] - row['hora']) <= 1800000, axis=1)]

resultado = resultado.rename(columns={'hora': 'hora_real'})
resultado = resultado.rename(columns={'aero': 'aero_real'})
resultado = resultado.rename(columns={'hora_formatted': 'hora_formatted_real'})



In [15]:
# MERGE COM RESULTADO E TC PREAV AGORA
tc_prev['aero'] = tc_prev['aero'].apply(lambda x: 'SB' + x)

In [16]:
resultado2 = resultado.merge(tc_prev, left_on=['destino'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado2 = resultado2[resultado2.apply(lambda row: abs(row['hora'] - row['dt_arr']) <= 1800000, axis=1)]
resultado2 = resultado2.rename(columns={'hora': 'hora_prev'})
resultado2 = resultado2.rename(columns={'hora_formatted': 'hora_formatted_prev'})
resultado2 = resultado2.rename(columns={'aero': 'aero_prev'})

In [17]:
## RESULTADO COM METAR
resultado3 = resultado2.merge(metar, left_on=['origem'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado3 = resultado3[resultado3.apply(lambda row: abs(row['hora'] - row['dt_dep']) <= 1800000, axis=1)]
resultado3 = resultado3.rename(columns={'hora': 'hora_metar'})
resultado3 = resultado3.rename(columns={'hora_formatted': 'hora_formatted_metar'})
resultado3 = resultado3.rename(columns={'aero': 'aero_metar'})


In [18]:
## RESULTADO COM METAF
resultado4 = resultado3.merge(metaf, left_on=['destino'], right_on=['aero'], how='inner')

# Use uma função lambda para calcular a diferença entre dt_dep e hora
resultado4 = resultado4[resultado4.apply(lambda row: abs(row['hora'] - row['dt_arr']) <= 1800000, axis=1)]
resultado4 = resultado4.rename(columns={'hora': 'hora_metaf'})
resultado4 = resultado4.rename(columns={'hora_formatted': 'hora_formatted_metaf'})
resultado4 = resultado4.rename(columns={'aero': 'aero_metaf'})


In [20]:
resultado4.head(1)

,flightid,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted,hora_real,nova_cabeceira,antiga_cabeceira,...,aero_prev,hora_formatted_prev,hora_metar,metar,aero_metar,hora_formatted_metar,hora_metaf,metaf,aero_metaf,hora_formatted_metaf
8025,b6fae10418d00900c1a8e8388af71abf,SBGR,SBBR,1679325532000,1679330517000,"Monday, March 20, 2023 12:18:52","Monday, March 20, 2023 13:41:57",1679326233000,28L,10R,...,SBBR,"Monday, March 20, 2023 14:00:00",1679324400000,METAR SBGR 201500Z 28006KT 9999 SCT045 30/18 Q...,SBGR,"Monday, March 20, 2023 12:00:00",1679331600000,METAF SBBR 201700Z 07008KT CAVOK 26/15 Q1013=\n,SBBR,"Monday, March 20, 2023 14:00:00"


In [ ]:
resultado4.to_csv("../data/processed_data/merged_bronze.csv")